In [1]:
import cv2
import os
import numpy as np

In [2]:
def load_templates(directory="cropped_captured_templates"):
    templates = {}
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):  # make sure to match your file extension
            name = filename.split('.')[0]
            img_path = os.path.join(directory, filename)
            template = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            templates[name] = template
    return templates

templates = load_templates()

In [3]:
def find_best_match(frame, templates):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    best_score = 0
    best_name = None
    best_loc = None
    template_size = (0, 0)  

    for name, template in templates.items():
        res = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        if max_val > best_score:
            best_score = max_val
            best_name = name
            best_loc = max_loc
            template_size = template.shape[::-1]  

    return best_name, best_score, best_loc, template_size

In [101]:
def calculate_orientation(contour):

    if len(contour) < 5:
        return 0
    (x, y), (MA, ma), angle = cv2.fitEllipse(contour)
    return angle

In [122]:
templates = load_templates()
cap = cv2.VideoCapture(0)
frame_history = []
capture_counter = 0

In [123]:
test_cases_directory = "test_cases"
if not os.path.exists(test_cases_directory):
    os.makedirs(test_cases_directory)

In [124]:
print("Press 's' to capture a frame, 'q' to quit.")

Press 's' to capture a frame, 'q' to quit.


In [125]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if len(frame_history) > 10:
        frame_history.pop(0)
    frame_history.append(gray_frame)
    
    if len(frame_history) == 10:
        motion_energy = np.zeros_like(gray_frame)
        for i in range(1, 10):
            motion_energy += cv2.absdiff(frame_history[i], frame_history[i-1])
        motion_energy = cv2.threshold(motion_energy, 50, 255, cv2.THRESH_BINARY)[1]
        
        contours, _ = cv2.findContours(motion_energy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            angle = calculate_orientation(contour)
            cv2.putText(frame, f'Angle: {angle:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
    
    best_match, score, loc, size = find_best_match(frame, templates)
    
    if score > 0.7:
        top_left = loc
        bottom_right = (top_left[0] + size[0], top_left[1] + size[1])
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
        cv2.putText(frame, f'Match: {best_match}, Score: {score:.2f}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    cv2.imshow("Real-time Template Matching", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        # Capture the current frame
        capture_path = os.path.join(test_cases_directory, f"captured_frame_{capture_counter}.jpg")
        cv2.imwrite(capture_path, frame)
        print(f"Frame captured and saved as {capture_path}.")
        capture_counter += 1
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Frame captured and saved as test_cases\captured_frame_0.jpg.
Frame captured and saved as test_cases\captured_frame_1.jpg.
Frame captured and saved as test_cases\captured_frame_2.jpg.
Frame captured and saved as test_cases\captured_frame_3.jpg.
Frame captured and saved as test_cases\captured_frame_4.jpg.
